In [1]:
import torch
from torch import nn, tensor, float32, stack

### Домашнее задание
**Разработчик: Тимофей Науменко**

На основании функции по варианту необходимо:
1. Построить nn.Module, в котором вы определите `forward`и `my_forward_backward`
2. `forward` должен повторять функцию, выданную вам по варианту, `my_forward_backward` описывает проход по вычислителному графу, а также вычисление градиентов по этому графу с помощью backprop. Градиенты должны быть рассчитаны для параметров $w0, w1$, тензоры $x1, x2, x3$ считаются входными данными сети
3. Необходимо удалять неиспользуемые тензоры, как это делает Pytorch
4. Если какие-то узлы не нужны для вычисления результата, то вы не должны их вычислять в процессе backprop
5. Необходимо построить визуализацию вычислительного графа

Все материалы для выполнения задания можно найти в ноутбуках второго и третьего семинаров, а также в этом ноутбуке есть пример выполненного задания

In [2]:
def v9():  # Alexander
  r1 = w0 * (x3 + w1 ** 2)
  r2 = (3 + x2) * w0 + w1
  r3 = x1 + (x2 * w0) / w1 ** 2
  return x1 - x2 / (r1 * r2 * r3 + x3)

In [3]:
def test_model_class(model_class):

    model = model_class()

    for _ in range(10):

        x1, x2, x3 = torch.rand(3)

        model.zero_grad()
        y_torch = model(x1, x2, x3)
        y_torch.backward()
        grad_torch = model.w.grad.clone()

        # print(y_torch.clone().detach().numpy(), grad_torch.clone().numpy())

        model.zero_grad()
        with torch.no_grad():
            y_manual = model.your_forward_backward(x1, x2, x3)
        grad_manual = model.w.grad.clone()

        # print(y_manual.numpy(), grad_manual.numpy())
        # print()

        assert torch.allclose(y_manual, y_torch, rtol=5e-05, atol=1e-7)
        assert torch.allclose(grad_manual, grad_torch, rtol=5e-05, atol=1e-7)

    print('Tests completed successfully!')

In [4]:
# Пример
class Graph(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(tensor([0.950, 0.288], dtype=float32))

    def forward(self, x1, x2, x3):
        w0, w1 = self.w

        r1 = w0 * (x3 + w1 ** 2)
        r2 = (3 + x2) * w0 + w1
        r3 = x1 + (x2 * w0) / w1 ** 2
        return x1 - x2 / (r1 * r2 * r3 + x3)

    def your_forward_backward(self, x1, x2, x3):
        w0, w1 = self.w

        # forward
        a0 = w1 ** 2
        a1 = x3 + a0
        a3 = w0 * a1

        b0 = 3 + x2
        b1 = b0 * w0
        b2 = b1 + w1
        b1 = None

        c1 = x2 * w0
        c2 = c1 / a0
        c3 = x1 + c2
        c2 = None

        d0 = a3 * b2 * c3
        d1 = d0 + x3
        x3 = None
        d0 = None
        d2 = x2 / d1
        d3 = x1 - d2
        x1 = None
        d2 = None

        # SyKa  # dlos
        dd3 = 1
        dd2 = dd3 * (-1)
        dd1 = dd2 * (-x2 / (d1 ** 2))
        d1 = None
        dd0 = dd1 * (1)

        dc3 = dd0 * (a3 * b2)
        dc2 = dc3 * (1)
        dc1 = dc2 * (1 / a0)

        db2 = dd0 * (a3 * c3)
        a3 = None
        db1 = db2 * (1)
        db0 = db1 * (w0)

        da3 = dd0 * (b2 * c3)
        c3 = None
        b2 = None
        da1 = da3 * (w0)
        da0 = da1 * (1) +dc2*(-c1/(a0**2))
        c1 = None
        a0 = None

        dw0 = da3 * (a1) + db1 * (b0) + dc1 * (x2)
        x2 = None
        b0 = None
        a1 = None
        dw1 = da0 * (2 * w1) + db2 * (1)

        self.w.grad = stack([dw0, dw1])

        return d3

In [5]:
# Здесь тестируется построенная вами модель
test_model_class(Graph)

Tests completed successfully!


In [6]:
# А сюда вы должны вставить визуализацию графа